リメイク版動画＃１

In [3]:
#インポート文まとめ

from bs4 import BeautifulSoup
import time
from tqdm.notebook import tqdm
from urllib.request import urlopen
import pandas as pd
import re
from selenium.webdriver.chrome.options import Options
import traceback
from selenium.webdriver.common.by import By
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import pickle
import scraping
import create_rawdf
from pathlib import Path
import create_rawdf

%load_ext autoreload

In [4]:
# 最初に実行
%autoreload

In [5]:
from urllib.request import urlopen
import pandas as pd

#url = "https://db.netkeiba.com/race/202304040509"
#html = urlopen(url).read()

#pd.read_html(html)[0]

2023年1月の開催日程取得　スクレイピング

In [6]:
from bs4 import BeautifulSoup
import re
#url = "https://race.netkeiba.com/top/calendar.html?year=2023&month=1"
#html = urlopen(url).read()

#soup = BeautifulSoup(html)
#a_list = soup.find("table",class_= "Calendar_Table").find_all("a")

#kaisai_date_list = []
#for a in a_list:
#    kaisai_date =re.findall(r"kaisai_date=(\d{8})",a["href"])[0]
#    kaisai_date_list.append(kaisai_date)




In [7]:
#引数に該当するカレンダーページを取得する関数
def scrape_kaisai_date(year_,month_):
    url = f"https://race.netkeiba.com/top/calendar.html?year={year_}&month={month_}"
    html = urlopen(url).read()
    soup = BeautifulSoup(html)
    a_list = soup.find("table",class_= "Calendar_Table").find_all("a")
    kaisai_date_list = []
    for a in a_list:
        kaisai_date =re.findall(r"kaisai_date=(\d{8})",a["href"])[0]
        kaisai_date_list.append(kaisai_date)
    return kaisai_date_list

#実行例
#scrape_kaisai_date(year_=2023,month_=1)    

In [8]:
#scrape_kaisai_date 年間の各月のレース日程を自動取得するように改変したい
def scrape_kaisai_date(from_,to_):
    url = f"https://race.netkeiba.com/top/calendar.html?year={year_}&month={month_}"
    html = urlopen(url).read()
    soup = BeautifulSoup(html)
    a_list = soup.find("table",class_= "Calendar_Table").find_all("a")
    kaisai_date_list = []
    for a in a_list:
        kaisai_date =re.findall(r"kaisai_date=(\d{8})",a["href"])[0]
        kaisai_date_list.append(kaisai_date)
    return kaisai_date_list


url = "https://race.netkeiba.com/top/calendar.html?year=2023&month=1"
html = urlopen(url).read()

2023年1月以外のレース日程も取得する

In [9]:
#年月を取得する方法
#date_rangeのfreq属性について　Mは月ごとに区切り、Sは月始めを指定する設定
from_="2023-01"
to_="2023-12"

for date in pd.date_range(from_,to_,freq="MS"):
    year = date.year
    month = date.month
    url = f"https://race.netkeiba.com/top/calendar.html?year={year}&month={month}"
    print(url)

https://race.netkeiba.com/top/calendar.html?year=2023&month=1
https://race.netkeiba.com/top/calendar.html?year=2023&month=2
https://race.netkeiba.com/top/calendar.html?year=2023&month=3
https://race.netkeiba.com/top/calendar.html?year=2023&month=4
https://race.netkeiba.com/top/calendar.html?year=2023&month=5
https://race.netkeiba.com/top/calendar.html?year=2023&month=6
https://race.netkeiba.com/top/calendar.html?year=2023&month=7
https://race.netkeiba.com/top/calendar.html?year=2023&month=8
https://race.netkeiba.com/top/calendar.html?year=2023&month=9
https://race.netkeiba.com/top/calendar.html?year=2023&month=10
https://race.netkeiba.com/top/calendar.html?year=2023&month=11
https://race.netkeiba.com/top/calendar.html?year=2023&month=12


In [10]:
#scrape_kaisai_date 年間の各月のレース日程を自動取得するように上記の年月取得方法を組み込み
import time
from tqdm.notebook import tqdm

def scrape_kaisai_date(from_,to_):
    kaisai_date_list = []
    for date in tqdm(pd.date_range(from_,to_,freq="MS")):
        year = date.year
        month = date.month
        url = f"https://race.netkeiba.com/top/calendar.html?year={year}&month={month}"
        html = urlopen(url).read()
        time.sleep(1) #1秒の感覚をあけてスクレイピングするように絶対やる　→　Webページへのアクセス負荷とならないような配慮
        soup = BeautifulSoup(html)
        a_list = soup.find("table",class_= "Calendar_Table").find_all("a")
        
        for a in a_list:
            kaisai_date =re.findall(r"kaisai_date=(\d{8})",a["href"])[0]
            kaisai_date_list.append(kaisai_date)
        
    return kaisai_date_list

In [17]:
scrape_kaisai_date(from_="2024-01",to_="2024-12")

  0%|          | 0/12 [00:00<?, ?it/s]

HTTPError: HTTP Error 400: Bad Request

ここまでの内容をscraping.pyに移行してます！！

リメイク版＃２
上記で取得した日程から今度はレースごとのURLを抽出する方法を考える

In [13]:
url = "https://race.netkeiba.com/top/race_list.html?kaisai_date=20230105"
html= urlopen(url).read()
soup = BeautifulSoup(html)

HTTPError: HTTP Error 400: Bad Request

In [65]:
#soup.find("div",class_="RaceList_DataItem")

上記だと取得できないのでChromeDriverを使用する（javaScriptで画面を動的に制御しているため）
ChromeDriverのため以下パッケージをインストールする

In [66]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [67]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [70]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

driver_path = ChromeDriverManager().install()

'C:\\Users\\hisaf\\.wdm\\drivers\\chromedriver\\win64\\130.0.6723.69\\chromedriver-win32/chromedriver.exe'

In [74]:
driver = webdriver.Chrome(service=Service(driver_path))

In [79]:
driver.get(url)

In [84]:
from selenium.webdriver.common.by import By

li_list = driver.find_elements(By.CLASS_NAME, "RaceList_DataItem")

In [85]:
li = li_list[0]

In [93]:
href = li.find_element(By.TAG_NAME,"a").get_attribute("href")
race_id = re.findall(r"race_id=(\d{12})", href)[0]

In [95]:
race_id_list = []
for li in li_list:
    href = li.find_element(By.TAG_NAME,"a").get_attribute("href")
    race_id = re.findall(r"race_id=(\d{12})", href)[0]
    # print(race_id)
    race_id_list.append(race_id)

In [97]:
driver.quit()

前回のソースコードとの連結を考える
関数化する

In [3]:
import scraping

kaisai_date_list = scraping.scrape_kaisai_date(from_="2023-01",to_="2023-12")

  0%|          | 0/12 [00:00<?, ?it/s]

In [9]:
from selenium.webdriver.chrome.options import Options
import traceback

def scrape_race_id_list(kaisai_date_list: list[str]):
    options = Options()
    options.add_argument("--headless")#バックグラウンドで実行
    driver_path = ChromeDriverManager().install()
    race_id_list = []

    with webdriver.Chrome(service=Service(driver_path),options=options) as driver:
        for kaisai_date in tqdm(kaisai_date_list):
            url = f"https://race.netkeiba.com/top/race_list.html?kaisai_date={kaisai_date}"
            try:
                driver.get(url)
                time.sleep(1)
                li_list = driver.find_elements(By.CLASS_NAME, "RaceList_DataItem")
                for li in li_list:
                    href = li.find_element(By.TAG_NAME,"a").get_attribute("href")
                    race_id = re.findall(r"race_id=(\d{12})", href)[0]
                    race_id_list.append(race_id)
            except:
                print(f"stopped at {url}")
                print(traceback.format_exc())
                break
    return race_id_list

In [6]:
race_id_list = scrape_race_id_list(kaisai_date_list)

  0%|          | 0/107 [00:00<?, ?it/s]

In [8]:
with open("race_id_list.pickle","wb") as f:
    pickle.dump(race_id_list,f)

リメイク版動画＃３

In [ ]:
url = "https://db.netkeiba.com/race/202306010101"
html = urlopen(url).read()
pd.read_html(html)[0]

In [117]:
import pickle

with open("race_id_list.pickle","rb") as f:
    race_id_list = pickle.load(f)

In [134]:
for race_id in race_id_list[:10]:
    url = f"https://db.netkeiba.com/race/{race_id}"
    html = urlopen(url).read()
    # print(url)

In [127]:
with open("data/html/202306010102.bin","wb") as f:
    f.write(html)

In [10]:
from pathlib import Path
HTML_DIR=Path("data","html")

In [165]:
HTML_DIR/"202306010101.bin"

WindowsPath('data/html/202306010101.bin')

In [166]:
with open(HTML_DIR/"202306010101.bin","wb") as f:
    f.write(html)

In [141]:
for race_id in race_id_list[:10]:
    url = f"https://db.netkeiba.com/race/{race_id}"
    html = urlopen(url).read()
    time.sleep(1)
    with open("data/html/"f"{race_id}.bin","wb") as f:
        f.write(html)


In [172]:
def scrape_html_race(race_id_list:list[str],save_dir:Path) -> list[Path]:
    html_path_list = []
    for race_id in tqdm(race_id_list):
        filepath = save_dir / f"{race_id}.bin"
        if filepath.is_file():
            print(f"skipped:{race_id}")
        else:
            url = f"https://db.netkeiba.com/race/{race_id}"
            html = urlopen(url).read()
            time.sleep(1)
            
            with open(filepath,"wb") as f:
                f.write(html)
            html_path_list.append(filepath)
    return html_path_list

In [5]:
html_path_list = scrape_html_race(race_id_list[:12],HTML_DIR)

NameError: name 'scrape_html_race' is not defined

In [4]:
HTML_RACE_DIR = HTML_DIR / "race"

NameError: name 'HTML_DIR' is not defined

In [3]:
html_paths_race = list(HTML_RACE_DIR.glob("*"))

NameError: name 'HTML_RACE_DIR' is not defined

In [1]:
dfs = {}
for html_path in html_paths_race:
    with open(html_path, "rb") as f:
        race_id = html_path.stem
        html = f.read()
        df = pd.read_html(html)[0]
        df.index = [race_id]*len(df)
        dfs[race_id] = df
concat_df = pd.concat(dfs.values())
concat_df.index.name = "race_id"

NameError: name 'html_paths_race' is not defined

In [2]:
pd.concat(dfs.values())

NameError: name 'pd' is not defined

In [29]:
def create_results(html_path_list: list[Path]) -> pd.DataFrame:
    dfs = {}
    for html_path in tqdm(html_paths_race):
        with open(html_path, "rb") as f:
            race_id = html_path.stem
            html = f.read()
            df = pd.read_html(html)[0]
            df.index = [race_id]*len(df)
            dfs[race_id] = df
    concat_df = pd.concat(dfs.values())
    concat_df.index.name = "race_id"
    return concat_df

In [31]:
results = create_results(html_path_list= html_paths_race)

  0%|          | 0/3456 [00:00<?, ?it/s]

In [32]:
results

,着 順,枠 番,馬 番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人 気,馬体重,調教師
race_id,,,,,,,,,,,,,
202301010101,1,5,5,サトミノキラリ,牡2,55.0,横山武史,1:09.5,NaN,1.2,1.0,452(-4),[東] 鈴木伸尋
202301010101,2,8,8,ベアゴーゴー,牝2,55.0,浜中俊,1:09.5,クビ,4.1,2.0,454(+2),[東] 杉浦宏昭
202301010101,3,6,6,ハピアーザンエバー,牡2,55.0,藤岡佑介,1:10.0,2.1/2,59.9,6.0,438(-6),[西] 羽月友彦
202301010101,4,4,4,デビルシズカチャン,牝2,55.0,ルメール,1:10.2,1.1/2,16.6,3.0,450(+2),[西] 武幸四郎
202301010101,5,1,1,ウィスピースノー,牝2,55.0,吉田隼人,1:10.3,1/2,23.9,5.0,434(-10),[西] 今野貞一
...,...,...,...,...,...,...,...,...,...,...,...,...,...
202310030812,8,1,1,リヴァーレ,牝3,53.0,太宰啓介,2:42.9,1.1/4,62.8,11.0,394(+2),[西] 松永幹夫
202310030812,9,6,7,シュアーウィナー,牡5,55.0,川端海翼,2:43.5,3.1/2,125.6,12.0,470(-4),[西] 安田隆行
202310030812,10,2,2,アウローラシエル,牡3,55.0,和田竜二,2:43.6,3/4,9.8,5.0,496(+8),[西] 森田直行


リメイク版動画＃４

In [33]:
with open("results.pickle","rb") as f:
    results = pickle.load(f)

In [35]:
url = "https://db.netkeiba.com/horse/2020103575"

In [36]:
html = urlopen(url).read()

In [ ]:
pd.read_html(html)[0]

In [11]:
url = "https://db.netkeiba.com/race/202306010101"
html = urlopen(url).read()
soup = BeautifulSoup(html,"lxml")

In [12]:
table = soup.find("table",class_="race_table_01 nk_tb_common")

In [6]:
#/horse/から始まるhref属性を持つaタグの一覧を取得する
a_list = table.find_all("a",href=re.compile(r"^/horse/"))

NameError: name 'table' is not defined

In [14]:
a = a_list[0]
a["href"]

'/horse/2020103575/'

In [76]:
re.findall(r"\d{10}",a["href"])[0]

'2020103575'

In [15]:
horse_id_list = []
for a in a_list:
    horse_id = re.findall(r"\d{10}",a["href"])[0]
    #print(horse_id)
    horse_id_list.append(horse_id)

In [16]:
horse_id_list

['2020103575',
 '2020107073',
 '2020102562',
 '2020106345',
 '2020100039',
 '2020104817',
 '2020104295',
 '2020105491',
 '2020104358',
 '2020105324',
 '2020103960',
 '2020102110',
 '2020102145',
 '2020101415',
 '2020106714',
 '2020100424']

In [6]:
HTML_RACE_DIR = Path("data","html","race")

In [17]:
html_paths_race = list(HTML_RACE_DIR.glob("*.bin"))
html_paths_race

[WindowsPath('data/html/race/202301010101.bin'),
 WindowsPath('data/html/race/202301010102.bin'),
 WindowsPath('data/html/race/202301010103.bin'),
 WindowsPath('data/html/race/202301010104.bin'),
 WindowsPath('data/html/race/202301010105.bin'),
 WindowsPath('data/html/race/202301010106.bin'),
 WindowsPath('data/html/race/202301010107.bin'),
 WindowsPath('data/html/race/202301010108.bin'),
 WindowsPath('data/html/race/202301010109.bin'),
 WindowsPath('data/html/race/202301010110.bin'),
 WindowsPath('data/html/race/202301010111.bin'),
 WindowsPath('data/html/race/202301010112.bin'),
 WindowsPath('data/html/race/202301010201.bin'),
 WindowsPath('data/html/race/202301010202.bin'),
 WindowsPath('data/html/race/202301010203.bin'),
 WindowsPath('data/html/race/202301010204.bin'),
 WindowsPath('data/html/race/202301010205.bin'),
 WindowsPath('data/html/race/202301010206.bin'),
 WindowsPath('data/html/race/202301010207.bin'),
 WindowsPath('data/html/race/202301010208.bin'),
 WindowsPath('data/h

In [33]:
results = create_rawdf.create_results(html_paths_race[:10])

  0%|          | 0/10 [00:00<?, ?it/s]

In [24]:
results.isnull().sum()

着 順            0
枠 番            0
馬 番            0
馬名             0
性齢             0
斤量             0
騎手             0
タイム            0
着差            10
単勝             0
人 気            0
馬体重            0
調教師            0
horse_id       0
jockey_id      0
trainer_id     0
owner_id       0
dtype: int64

In [26]:
results.columns = results.columns.str.replace(" ","")

In [27]:
results.columns = results.columns.str.replace(" ","")

Index(['着順', '枠番', '馬番', '馬名', '性齢', '斤量', '騎手', 'タイム', '着差', '単勝', '人気',
       '馬体重', '調教師', 'horse_id', 'jockey_id', 'trainer_id', 'owner_id'],
      dtype='object')

In [28]:
save_dir = Path("data","rawdf")
save_dir.mkdir()

In [31]:
results.to_csv(save_dir/"results.csv",sep="\t")

In [37]:
horse_id_list = results["horse_id"].unique()

In [38]:
len(horse_id_list)

118

In [43]:
html_paths_horse = scraping.scrape_html_horse(horse_id_list= horse_id_list)

  0%|          | 0/118 [00:00<?, ?it/s]

In [45]:
horse_results = create_rawdf.create_horse_results(html_path_list = html_paths_horse)

  0%|          | 0/118 [00:00<?, ?it/s]

In [46]:
horse_results

,日付,開催,天気,R,レース名,映像,頭数,枠番,馬番,オッズ,...,着差,ﾀｲﾑ指数,通過,ペース,上り,馬体重,厩舎ｺﾒﾝﾄ,備考,勝ち馬(2着馬),賞金
horse_id,,,,,,,,,,,,,,,,,,,,,
2021101429,2024/10/06,4東京2,曇,8,3歳以上2勝クラス,NaN,13,6,8,1.9,...,-0.5,**,6-6,33.8-34.5,33.8,456(-2),NaN,NaN,(ニコラウス),1140.0
2021101429,2024/09/01,3新潟8,晴,9,飯豊特別(2勝クラス),NaN,7,7,7,3.0,...,0.4,**,4-4,35.2-34.4,34.4,458(-2),NaN,NaN,セントメモリーズ,625.2
2021101429,2024/08/10,3新潟1,晴,10,新発田城特別(2勝クラス),NaN,14,8,13,11.3,...,0.0,**,10-10,33.5-35.0,33.9,460(-4),NaN,NaN,サクセスアイ,629.2
2021101429,2024/04/06,3中山5,曇,11,ニュージーランドT(GII),NaN,16,8,16,75.3,...,0.9,**,14-16-16,36.0-35.5,34.9,464(0),NaN,NaN,エコロブルーム,NaN
2021101429,2024/03/16,1中京3,晴,11,中スポ賞ファルコンS(GIII),NaN,17,2,3,10.1,...,0.9,**,12-12,33.8-34.7,34.7,464(+8),NaN,NaN,ダノンマッキンリー,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018105226,2021/02/13,1東京5,晴,7,3歳1勝クラス,NaN,16,1,1,20.2,...,0.7,**,4-3,34.4-35.5,35.9,454(+8),NaN,NaN,レイモンドバローズ,NaN
2018105226,2020/11/22,5東京6,晴,3,2歳未勝利,NaN,18,7,14,9.7,...,0.0,**,2-2,35.6-34.6,34.4,446(0),NaN,NaN,(アップリバー),510.0
2018105226,2020/11/01,3福島2,晴,2,2歳未勝利,NaN,15,1,1,1.8,...,0.1,**,4-3,34.0-35.8,35.6,446(-2),NaN,NaN,テセウス,130.0


リメイク版動画＃6 レース情報の取得

In [1]:
#2/1実行
from pathlib import Path
import pandas as pd
from bs4 import BeautifulSoup
import re

DATA_DIR = Path("..","data")
HTML_DIR = DATA_DIR / "html"
HTML_RACE_DIR = HTML_DIR / "race"

In [2]:
#2/1実行
html_path_list = list(HTML_RACE_DIR.glob("*.bin"))


In [3]:
#2/1実行
html_path_list[0]

WindowsPath('../data/html/race/202201010101.bin')

In [5]:
#2/1実行
with open(html_path_list[0],"rb") as f:
    html = f.read()

In [6]:
#2/1実行
html

b'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml" lang="ja" xml:lang="ja" id="html">\n<head>\n\n<title>2\xba\xd0\xcc\xa4\xbe\xa1\xcd\xf8\xa1\xc32022\xc7\xaf7\xb7\xee23\xc6\xfc | \xb6\xa5\xc7\xcf\xa5\xc7\xa1\xbc\xa5\xbf\xa5\xd9\xa1\xbc\xa5\xb9 - netkeiba</title>\n\n\n<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n<meta http-equiv="content-language" content="ja" />\n<meta http-equiv="content-type" content="text/html; charset=euc-jp" />\n<meta http-equiv="content-script-type" content="text/javascript" />\n<meta http-equiv="content-style-type" content="text/css" />\n<meta name="description" content="netkeiba\xa4\xac\xb8\xd8\xa4\xeb\xb9\xf1\xc6\xe2\xba\xc7\xc2\xe7\xb5\xe9\xa4\xce\xb6\xa5\xc7\xcf\xa5\xc7\xa1\xbc\xa5\xbf\xa5\xd9\xa1\xbc\xa5\xb9\xa4\xc7\xa4\xb9\xa1\xa350\xcb\xfc\xc6\xac\xb0\xca\xbe\xe5\xa4\xce\xb6\xa5\xc1\xf6\xc7\xcf\xa1\xa2\xb5\xb3\xbc\xea\xa1

In [ ]:
#2/1実行
#htmlを加工できる状態にする
soup = BeautifulSoup(html,"lxml")
soup

In [11]:
#2/1実行
soup_info = soup.find("div",class_="data_intro")
soup_info.find("h1").text

'2歳未勝利'

In [12]:
#2/1実行
tmp = soup_info.find("p").text.replace(" ","")
tmp

'\n\n芝右1800m\xa0/\xa0天候:曇\xa0/\xa0芝:稍重\xa0/\xa0発走:09:50\n\n\n'

In [13]:
#2/1実行
#文字化けを取り除く
re.findall(r"[\w:]+",tmp)

['芝右1800m', '天候:曇', '芝:稍重', '発走:09:50']

In [14]:
#2/1実行
tmp = soup_info.find_all("p")[1].text
tmp

'2022年7月23日 1回札幌1日目 2歳未勝利\xa0\xa0(混)(馬齢)'

In [15]:
#2/1実行
re.findall(r"\w+",tmp)

['2022年7月23日', '1回札幌1日目', '2歳未勝利', '混', '馬齢']

In [16]:
#2/1実行
info_dict = {}
info_dict["title"] = soup_info.find("h1").text
info_dict["info1"] = re.findall(
    r"[\w:]+",soup_info.find("p").text.replace(" ","")
)
info_dict["info2"] = re.findall(r"\w+",soup_info.find_all("p")[1].text)
info_dict

{'title': '2歳未勝利',
 'info1': ['芝右1800m', '天候:曇', '芝:稍重', '発走:09:50'],
 'info2': ['2022年7月23日', '1回札幌1日目', '2歳未勝利', '混', '馬齢']}

In [17]:
#2/1実行
pd.DataFrame().from_dict(info_dict,orient="index").T

,title,info1,info2
0,2歳未勝利,"[芝右1800m, 天候:曇, 芝:稍重, 発走:09:50]","[2022年7月23日, 1回札幌1日目, 2歳未勝利, 混, 馬齢]"


In [ ]:
#上記はすべて「create_rawdf.py（create_race_info）」にまとめてる

In [19]:
#2/1実行
import create_rawdf

race_info = create_rawdf.create_race_info(html_path_list)

  0%|          | 0/10606 [00:00<?, ?it/s]

'NoneType' object has no attribute 'find' at 202405050701
'NoneType' object has no attribute 'find' at 202405050702
'NoneType' object has no attribute 'find' at 202405050703
'NoneType' object has no attribute 'find' at 202405050704
'NoneType' object has no attribute 'find' at 202405050705
'NoneType' object has no attribute 'find' at 202405050706
'NoneType' object has no attribute 'find' at 202405050707
'NoneType' object has no attribute 'find' at 202405050708
'NoneType' object has no attribute 'find' at 202405050709
'NoneType' object has no attribute 'find' at 202405050710
'NoneType' object has no attribute 'find' at 202405050711
'NoneType' object has no attribute 'find' at 202405050712
'NoneType' object has no attribute 'find' at 202405050801
'NoneType' object has no attribute 'find' at 202405050802
'NoneType' object has no attribute 'find' at 202405050803
'NoneType' object has no attribute 'find' at 202405050804
'NoneType' object has no attribute 'find' at 202405050805
'NoneType' obj

In [21]:
#2/1実行
race_info

,race_id,title,info1,info2
0,202201010101,2歳未勝利,"[芝右1800m, 天候:曇, 芝:稍重, 発走:09:50]","[2022年7月23日, 1回札幌1日目, 2歳未勝利, 混, 馬齢]"
0,202201010102,3歳未勝利,"[ダ右1700m, 天候:曇, ダート:稍重, 発走:10:25]","[2022年7月23日, 1回札幌1日目, 3歳未勝利, 馬齢]"
0,202201010103,3歳未勝利,"[芝右1500m, 天候:曇, 芝:稍重, 発走:10:55]","[2022年7月23日, 1回札幌1日目, 3歳未勝利, 牝, 馬齢]"
0,202201010104,3歳未勝利,"[芝右1200m, 天候:曇, 芝:稍重, 発走:11:25]","[2022年7月23日, 1回札幌1日目, 3歳未勝利, 混, 馬齢]"
0,202201010105,2歳新馬,"[ダ右1700m, 天候:曇, ダート:稍重, 発走:12:15]","[2022年7月23日, 1回札幌1日目, 2歳新馬, 混, 馬齢]"
...,...,...,...,...
0,202510010208,4歳以上1勝クラス,"[ダ右2400m, 天候:晴, ダート:良, 発走:13:35]","[2025年1月26日, 1回小倉2日目, 4歳以上1勝クラス, 指, 定量]"
0,202510010209,八幡特別(1勝),"[芝右1200m, 天候:晴, 芝:良, 発走:14:05, 過去の八幡特別]","[2025年1月26日, 1回小倉2日目, 4歳以上1勝クラス, 混, 指, 定量]"
0,202510010210,海の中道特別(2勝),"[芝右2600m, 天候:晴, 芝:良, 発走:14:40, 過去の海の中道特別]","[2025年1月26日, 1回小倉2日目, 4歳以上2勝クラス, 混, 特指, 定量]"
0,202510010211,壇之浦ステークス(3勝),"[芝右1800m, 天候:晴, 芝:良, 発走:15:15, 過去の壇之浦ステークス]","[2025年1月26日, 1回小倉2日目, 4歳以上3勝クラス, 混, 特指, 定量]"


In [14]:
# 最初に実行
%autoreload

In [2]:
COMMON_DATA_DIR = Path("..", "..", "common","src","data")
INPUT_DIR = COMMON_DATA_DIR / "rawdf"
MAPPING_DIR = COMMON_DATA_DIR / "mapping"
OUTPUT_DIR = Path("..", "data", "01_preprocessed")
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

In [3]:
input_dir = INPUT_DIR
save_filename = "race_info.csv"

In [61]:
df = pd.read_csv(input_dir / save_filename, sep="\t")

In [18]:
(df["info2"].str.extract(r"(\d{4}年\d{1,2}月\d{1,2}日)"))

,0
0,2023年7月22日
1,2023年7月22日
2,2023年7月22日
3,2023年7月22日
4,2023年7月22日
...,...
3451,2023年9月3日
3452,2023年9月3日
3453,2023年9月3日
3454,2023年9月3日


In [62]:
#df["date"] = (df["info2"].str.extract(r"(\d+年\d+月\d+日)")).replace(r"(\D)(?=.{1}$)","-",regex=True)
# df["date2"]=df["date"].str.replace(r"\D","-",regex=True)
df["date"] = df["info2"].str.extract(r"(\d{4}年\d{1,2}月\d{1,2}日)")
df["date"] = pd.to_datetime(df["date"], format='%Y年%m月%d日').dt.strftime('%Y-%m-%d')

df["date"]

0       2023-07-22
1       2023-07-22
2       2023-07-22
3       2023-07-22
4       2023-07-22
           ...    
3451    2023-09-03
3452    2023-09-03
3453    2023-09-03
3454    2023-09-03
3455    2023-09-03
Name: date, Length: 3456, dtype: object

In [40]:
df["info1"]

0                  ['芝右1200m', '天候:晴', '芝:良', '発走:09:50']
1                ['ダ右1000m', '天候:晴', 'ダート:良', '発走:10:25']
2                ['ダ右1700m', '天候:晴', 'ダート:良', '発走:10:55']
3                  ['芝右1500m', '天候:晴', '芝:良', '発走:11:25']
4                ['ダ右1700m', '天候:晴', 'ダート:良', '発走:12:15']
                              ...                        
3451             ['ダ右1000m', '天候:晴', 'ダート:良', '発走:13:55']
3452    ['芝右1200m', '天候:晴', '芝:良', '発走:14:25', '過去の西日本...
3453    ['ダ右1700m', '天候:晴', 'ダート:良', '発走:15:01', '過去の宮...
3454    ['芝右1200m', '天候:晴', '芝:良', '発走:15:35', '過去の小倉2...
3455               ['芝右2600m', '天候:晴', '芝:良', '発走:16:15']
Name: info1, Length: 3456, dtype: object

#6のrace_infoの前処理関数忘れるな！！

In [50]:
df

,race_id,title,info1,info2,date,cource_info
0,202301010101,2歳未勝利,"['芝右1200m', '天候:晴', '芝:良', '発走:09:50']","['2023年7月22日', '1回札幌1日目', '2歳未勝利', '混', '指', '...",2023-07-22,芝右1200m
1,202301010102,3歳未勝利,"['ダ右1000m', '天候:晴', 'ダート:良', '発走:10:25']","['2023年7月22日', '1回札幌1日目', '3歳未勝利', '指', '馬齢']",2023-07-22,芝右1200m
2,202301010103,3歳未勝利,"['ダ右1700m', '天候:晴', 'ダート:良', '発走:10:55']","['2023年7月22日', '1回札幌1日目', '3歳未勝利', '指', '馬齢']",2023-07-22,芝右1200m
3,202301010104,3歳未勝利,"['芝右1500m', '天候:晴', '芝:良', '発走:11:25']","['2023年7月22日', '1回札幌1日目', '3歳未勝利', '牝', '指', '...",2023-07-22,芝右1200m
4,202301010105,2歳新馬,"['ダ右1700m', '天候:晴', 'ダート:良', '発走:12:15']","['2023年7月22日', '1回札幌1日目', '2歳新馬', '混', '指', '馬齢']",2023-07-22,芝右1200m
...,...,...,...,...,...,...
3451,202310030808,3歳以上1勝クラス,"['ダ右1000m', '天候:晴', 'ダート:良', '発走:13:55']","['2023年9月3日', '3回小倉8日目', '3歳以上1勝クラス', '指', '定量']",2023-09-03,芝右1200m
3452,202310030809,西日本新聞杯(2勝),"['芝右1200m', '天候:晴', '芝:良', '発走:14:25', '過去の西日本...","['2023年9月3日', '3回小倉8日目', '3歳以上2勝クラス', '混', '指'...",2023-09-03,芝右1200m
3453,202310030810,宮崎ステークス(3勝),"['ダ右1700m', '天候:晴', 'ダート:良', '発走:15:01', '過去の宮...","['2023年9月3日', '3回小倉8日目', '3歳以上3勝クラス', '混', '特指...",2023-09-03,芝右1200m
3454,202310030811,第43回小倉2歳ステークス(GIII),"['芝右1200m', '天候:晴', '芝:良', '発走:15:35', '過去の小倉2...","['2023年9月3日', '3回小倉8日目', '2歳オープン', '国際', '特指',...",2023-09-03,芝右1200m


In [16]:
#df["info1"].str.split(",")
df["info1"].str.extract(r"([^\s,]+)")

NameError: name 'df' is not defined

In [80]:
df["cource_info"]=(df["info1"].str.extract(r"([^\s,]+)"))
df["cource_info"] = df["cource_info"].str.extract(r"'([^']*)'")
df["cource_info"]

0       芝右1200m
1       ダ右1000m
2       ダ右1700m
3       芝右1500m
4       ダ右1700m
         ...   
3451    ダ右1000m
3452    芝右1200m
3453    ダ右1700m
3454    芝右1200m
3455    芝右2600m
Name: cource_info, Length: 3456, dtype: object

In [89]:
df["race_type"] = df["cource_info"].str.extract(r"([^\d{4}]+)")
df["race_type"].unique()

array(['芝右', 'ダ右', '障芝', '障芝外', 'ダ左', '芝左', '芝直線', '芝左外', '障芝ダート', '芝右外',
       '芝右内', '障芝内'], dtype=object)

In [96]:
df["tmp"] = df["info1"].map(lambda x: eval(x)[0])
df["tmp"]

0       芝右1200m
1       ダ右1000m
2       ダ右1700m
3       芝右1500m
4       ダ右1700m
         ...   
3451    ダ右1000m
3452    芝右1200m
3453    ダ右1700m
3454    芝右1200m
3455    芝右2600m
Name: tmp, Length: 3456, dtype: object

In [92]:
df["cource_len"] = df["cource_info"].str.extract(r"(\d{4})")

In [93]:
df["cource_len"].unique()

array(['1200', '1000', '1700', '1500', '1800', '2000', '2600', '2400',
       '2750', '1150', '3380', '2770', nan, '1600', '1400', '2200',
       '2500', '2100', '1300', '3400', '2300', '3000', '3110', '3200',
       '2880', '3350', '4250', '3210', '3570', '3600', '4100', '1900',
       '3300', '3330', '3930', '2910', '3170', '2970', '3900', '3140',
       '2860', '3390'], dtype=object)

In [86]:
df["cource_info"].unique()

array(['芝右1200m', 'ダ右1000m', 'ダ右1700m', '芝右1500m', '芝右1800m', '芝右2000m',
       '芝右2600m', 'ダ右2400m', '芝右1000m', '障芝2750m', 'ダ右1150m', '障芝3380m',
       '障芝2770m', '障芝外', 'ダ左1800m', 'ダ左1200m', '芝左2000m', '芝直線1000m',
       '芝左外1600m', '芝左1200m', '芝左外1800m', '芝左1400m', '芝左2400m', '芝左2200m',
       '芝左外2000m', 'ダ左2500m', 'ダ左1400m', 'ダ左1600m', '芝左1800m', '芝左1600m',
       'ダ左2100m', 'ダ左1300m', '芝左3400m', '芝左2300m', '障芝ダート3000m',
       '芝左2500m', '障芝3110m', 'ダ右1200m', 'ダ右1800m', '芝右外1600m',
       '障芝ダート3200m', '障芝ダート2880m', '芝右外2200m', '芝右外1200m', '芝右2500m',
       '障芝外3350m', '障芝外4250m', '障芝3210m', '障芝3570m', '芝右内2周3600m',
       'ダ右2500m', '障芝4100m', 'ダ左1900m', '芝左3000m', '障芝3000m', '障芝3300m',
       '障芝3330m', 'ダ右1900m', '芝右外2400m', 'ダ右1400m', '芝右外1800m', '芝右1600m',
       '芝右外1400m', '芝右1400m', '芝右外3200m', '障芝3930m', '障芝ダート2910m',
       '障芝ダート3170m', '芝右外3000m', '障芝内', 'ダ右2000m', '芝右2200m', '芝右3000m',
       '障芝ダート2970m', '障芝3900m', '芝右外2600m', '障芝ダート3110m', '障芝3140m',
       '障芝286

リメイク版　＃９

### 予測母集団の作成
予測したい日の（日付,race_id,horse_id）のペアを作る  

In [22]:
#2/1実行
import scraping
%load_ext autoreload

In [ ]:
#2/1実行 予測したい日のracd_id一覧を取得
race_id_list = scraping.scrape_race_id_list(["20250201"])

100%|██████████| 1/1 [00:35<00:00, 35.56s/it]


In [ ]:
#2/1実行
race_id_list

In [25]:
#2/1実行
len(race_id_list)

36

In [26]:
#2/1実行
#予測したい日の出馬をとってくる
#舞鶴Ｓ

from urllib.request import urlopen,Request

url = "https://race.netkeiba.com/race/shutuba.html?race_id=202508010111"

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"}
req = Request(url, headers=headers)

html = urlopen(req).read() #スクレイピング

In [27]:
#2/1実行
html

b'<!DOCTYPE html>\n<html>\n<head>\n<meta charset="EUC-JP">\n\n<!-- block=common__meta_tag_each_race (cp) -->\n<meta http-equiv="content-language" content="ja">\n<meta http-equiv="X-UA-Compatible" content="IE=edge"/>\n<meta name="viewport" content="width=1000">\n<meta name="format-detection" content="telephone=no" />\n<meta name="description" content="2025\xc7\xaf2\xb7\xee1\xc6\xfc \xb5\xfe\xc5\xd411R \xc9\xf1\xc4\xe1\xa5\xb9\xa5\xc6\xa1\xbc\xa5\xaf\xa5\xb9(3\xbe\xa1\xa5\xaf\xa5\xe9\xa5\xb9)\xa4\xce\xbd\xd0\xc7\xcf\xc9\xbd\xa4\xc7\xa4\xb9\xa1\xa3JRA\xb3\xab\xba\xc5\xa5\xec\xa1\xbc\xa5\xb9\xa4\xce\xbd\xd0\xc7\xcf\xc9\xbd\xa4\xe4\xba\xc7\xbf\xb7\xa5\xaa\xa5\xc3\xa5\xba\xa1\xa2\xa5\xec\xa1\xbc\xa5\xb9\xb7\xeb\xb2\xcc\xc2\xae\xca\xf3\xa1\xa2\xca\xa7\xcc\xe1\xbe\xf0\xca\xf3\xa4\xf2\xa4\xcf\xa4\xb8\xa4\xe1\xa1\xa2\xb6\xa5\xc7\xcf\xcd\xbd\xc1\xdb\xa4\xe4\xa5\xc7\xa1\xbc\xa5\xbf\xca\xac\xc0\xcf\xa4\xca\xa4\xc9\xcd\xbd\xc1\xdb\xa4\xcb\xcc\xf2\xce\xa9\xa4\xc4\xbe\xf0\xca\xf3\xa4\xe2\xcb\xfe\xba\x

In [28]:
#2/1実行　とってきたhtmlを加工
from bs4 import BeautifulSoup
soup = BeautifulSoup(html)
soup

<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8"/>
<!-- block=common__meta_tag_each_race (cp) -->
<meta content="ja" http-equiv="content-language"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=1000" name="viewport"/>
<meta content="telephone=no" name="format-detection"/>
<meta content="2025年2月1日 京都11R 舞鶴ステークス(3勝クラス)の出馬表です。JRA開催レースの出馬表や最新オッズ、レース結果速報、払戻情報をはじめ、競馬予想やデータ分析など予想に役立つ情報も満載です。" name="description"/>
<meta content="競馬,keiba,出馬表,オッズ,予想,レース結果,払戻し,結果R報,競馬?想,ネット競馬,netkeiba" name="keywords"/>
<meta content="https://www.netkeiba.com/style/netkeiba.ja/image/netkeiba.png" name="thumbnail"/>
<!-- ogp用 -->
<meta content="netkeiba" property="og:site_name"/>
<meta content="article" property="og:type"/>
<meta content="舞鶴ステークス(3勝クラス) 出馬表 | 2025年2月1日 京都11R レース情報(JRA) - netkeiba" property="og:title"/>
<meta content="https://race.netkeiba.com/race/shutuba.html?race_id=202508010111" property="og:url"/>
<meta content="2025年2月1日 京都11R 舞鶴ステークス(3勝クラス)の出馬表です。JRA開催レースの出

In [29]:
#2/1実行
td_list = soup.find_all("td", class_="HorseInfo")
td_list

[<td class="HorseInfo">
 <div>
 <div>
 <span class="HorseName"><a href="https://db.netkeiba.com/horse/2021100294" target="_blank" title="ネバーモア">ネバーモア<img alt="" class="disp_none Favorite" id="myhorse_2021100294" src="https://cdn.netkeiba.com/img.race/common/img/icon/icon_horse.png?2019073001" width="18"/></a></span>
 <img class="disp_none horsebookmark myhorse_2021100294" id="myhorse1_2021100294" onmouseout="this.src='https://cdn.netkeiba.com/img.race/common/img/icon/horse_favorite_icon_pc.png'" onmouseover="this.src='https://cdn.netkeiba.com/img.race/common/img/icon/horse_favorite_icon_pc.png'" src="https://cdn.netkeiba.com/img.race/common/img/icon/horse_favorite_icon_pc.png" width="20"/>
 </div>
 </div>
 </td>,
 <td class="HorseInfo">
 <div>
 <div>
 <span class="HorseName"><a href="https://db.netkeiba.com/horse/2021105569" target="_blank" title="サトノアイオライト">サトノアイオライト<img alt="" class="disp_none Favorite" id="myhorse_2021105569" src="https://cdn.netkeiba.com/img.race/common/img/icon/ic

In [30]:
#2/1実行
td = td_list[0]
td

<td class="HorseInfo">
<div>
<div>
<span class="HorseName"><a href="https://db.netkeiba.com/horse/2021100294" target="_blank" title="ネバーモア">ネバーモア<img alt="" class="disp_none Favorite" id="myhorse_2021100294" src="https://cdn.netkeiba.com/img.race/common/img/icon/icon_horse.png?2019073001" width="18"/></a></span>
<img class="disp_none horsebookmark myhorse_2021100294" id="myhorse1_2021100294" onmouseout="this.src='https://cdn.netkeiba.com/img.race/common/img/icon/horse_favorite_icon_pc.png'" onmouseover="this.src='https://cdn.netkeiba.com/img.race/common/img/icon/horse_favorite_icon_pc.png'" src="https://cdn.netkeiba.com/img.race/common/img/icon/horse_favorite_icon_pc.png" width="20"/>
</div>
</div>
</td>

In [31]:
#2/1実行
import re

re.findall(r"\d{10}", td.find("a")["href"])[0]

'2021100294'

In [32]:
#2/1実行
import create_prediction_population
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
%autoreload

In [38]:
#2/1実行
race_id_list[0]

'202505010101'

In [39]:
#2/1実行
create_prediction_population.scrape_horse_id_list(race_id_list[0])

['2022103340',
 '2022104847',
 '2022107196',
 '2022106071',
 '2022104523',
 '2022105403',
 '2022104967',
 '2022110064',
 '2022100485',
 '2022110025',
 '2022103259',
 '2022107130',
 '2022103556',
 '2022103030',
 '2022104095',
 '2022100749']

In [40]:
#2/1実行
#想定と違うところにファイルができてるから、パスの設定変かも
prediction_population = create_prediction_population.create(kaisai_date="20250201")

scraping race_id_list...


100%|██████████| 1/1 [00:13<00:00, 13.21s/it]


scraping horse_id_list...


  0%|          | 0/36 [00:00<?, ?it/s]

In [41]:
#2/1実行
prediction_population

,date,race_id,horse_id
0,2025-02-01,202505010101,2022103340
1,2025-02-01,202505010101,2022104847
2,2025-02-01,202505010101,2022107196
3,2025-02-01,202505010101,2022106071
4,2025-02-01,202505010101,2022104523
...,...,...,...
13,2025-02-01,202510010312,2021102054
14,2025-02-01,202510010312,2021105175
15,2025-02-01,202510010312,2020102537
16,2025-02-01,202510010312,2020103261


## 馬の過去成績の取得

In [42]:
#2/1実行
html_path_horse = scraping.scrape_html_horse(
    horse_id_list=prediction_population["horse_id"].unique(),
    skip=False
)

100%|██████████| 485/485 [11:13<00:00,  1.39s/it]


In [43]:
#2/1実行
html_path_horse

[WindowsPath('../data/html/horse/2022103340.bin'),
 WindowsPath('../data/html/horse/2022104847.bin'),
 WindowsPath('../data/html/horse/2022107196.bin'),
 WindowsPath('../data/html/horse/2022106071.bin'),
 WindowsPath('../data/html/horse/2022104523.bin'),
 WindowsPath('../data/html/horse/2022105403.bin'),
 WindowsPath('../data/html/horse/2022104967.bin'),
 WindowsPath('../data/html/horse/2022110064.bin'),
 WindowsPath('../data/html/horse/2022100485.bin'),
 WindowsPath('../data/html/horse/2022110025.bin'),
 WindowsPath('../data/html/horse/2022103259.bin'),
 WindowsPath('../data/html/horse/2022107130.bin'),
 WindowsPath('../data/html/horse/2022103556.bin'),
 WindowsPath('../data/html/horse/2022103030.bin'),
 WindowsPath('../data/html/horse/2022104095.bin'),
 WindowsPath('../data/html/horse/2022100749.bin'),
 WindowsPath('../data/html/horse/2022100949.bin'),
 WindowsPath('../data/html/horse/2022106754.bin'),
 WindowsPath('../data/html/horse/2022101156.bin'),
 WindowsPath('../data/html/hors

In [44]:
#2/1実行
import create_rawdf

horse_results_prediction = create_rawdf.create_horse_results(
    html_path_list= html_path_horse,
    save_filename= "horse_results_prediction.csv"
)

  0%|          | 0/485 [00:00<?, ?it/s]

table not found at 2022104523
table not found at 2022103259
table not found at 2022107130
table not found at 2022101156
table not found at 2022105502
table not found at 2022100624
table not found at 2022104561
table not found at 2022105892
table not found at 2022101200
table not found at 2022101741
table not found at 2022106543
table not found at 2022100464
table not found at 2022107042
table not found at 2022107154
table not found at 2022102526
table not found at 2022102830
table not found at 2022109048
table not found at 2022104895
table not found at 2022101071
table not found at 2022105398
table not found at 2022102300
table not found at 2022101176
table not found at 2022105324
table not found at 2022103934
table not found at 2022106174
table not found at 2022106946
table not found at 2022102515
table not found at 2022110005
table not found at 2022104448
table not found at 2022100055
table not found at 2022103505
table not found at 2022103453
table not found at 2022104391
table not 